In [12]:
import numpy as np
import tensorflow as tf

from pureEM import *


import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%reload_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# PM task ideas

- network given the PM cue on first trial, must respond with PM action
    - on first trial encode stim_rep(PMcue):cell_state
    - retrieval gate controled by sensory info and cell
    - PM cue presented unifrom random after trial 25
    - trainable initial cell state
    - episode unique (random) PM cue
- goal of this first test is to see if retrieval gate is learnable
    - can network shut off retrieval during ongoing task?
- since during training I am giving the network feedback on every stimulus, how do I prevent it from just learning a mapping from stimulus to output in cell state? 
- maybe the first thing to check is pureWM model error rate on PM trials increase with OG task difficulty. then can I mitigate this with an EM architecture?

- 2back and 3back in same episode on random vectors 
    - DND has two slots corresponding to 2back and 3back
    - encoding happens when task shifts
    - learn to retrieve relevant cell state when task shifts